In [1]:
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa
import tqdm
from tensorflow import keras
from scipy import stats

In [4]:
df = pd.read_csv("/home/aimlproject1/home/soft_cluster_comp.csv")
df.shape

(3579, 12)

In [2]:
# df3 = pd.read_csv("/home/aimlproject1/home/dfs/3.csv")
# s3 = df3
# print("3")
# df4 = pd.read_csv("/home/aimlproject1/home/dfs/4.csv")
# s4 = df4
# print("4")
# df5 = pd.read_csv("/home/aimlproject1/home/dfs/5.csv")
# s5 = df5
# print("5")
# df6 = pd.read_csv("/home/aimlproject1/home/dfs/6.csv")
# s6 = df6
# print("6")
# df7 = pd.read_csv("/home/aimlproject1/home/dfs/7.csv")
# s7 = df7
# print("7")
df8 = pd.read_csv("/home/aimlproject1/home/dfs/8.csv")
s8 = df8
print("8")
df9 = pd.read_csv("/home/aimlproject1/home/dfs/9.csv")
s9 = df9
print("9")
# df0.head()

8
9


In [3]:
df = ["d", "r" ,"df2" , 'df3' , 'df4', 'df5' , 'df6' , "df7" , df8 , df9]
def get_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation="swish")(investment_id_x)
    investment_id_x = layers.BatchNormalization()(investment_id_x)
    investment_id_x = layers.Dense(64, activation="swish")(investment_id_x)
    investment_id_x = layers.BatchNormalization()(investment_id_x)
    investment_id_x = layers.Dense(64, activation="swish")(investment_id_x)
    investment_id_x = layers.BatchNormalization()(investment_id_x)
    # "swish" tfa.activations.lisht
    feature_x = layers.Dense(256, activation="swish")(features_inputs)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.Dense(256, activation="swish")(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.Dense(256, activation="swish")(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x,feature_x])
    x = layers.Dense(512, activation="swish", kernel_regularizer="l2")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation="swish", kernel_regularizer="l2")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(32, activation="swish", kernel_regularizer="l2")(x)
    x = layers.BatchNormalization()(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model


In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_dir = [
 ['ensemle_0_model_1'],
  ['ensemle_1_model_1'],
   ['ensemle_2_model_1'],
    ['ensemle_3_model_1'],
     ['ensemle_4_model_1'],
     ['ensemle_5_model_1'],
     ['ensemle_6_model_1'],
     ['ensemle_7_model_1'],
     ['ensemle_8_model_1'],
     ['ensemle_9_model_1']
     ]
checkpoints = []
for i in  range(0,len(checkpoint_dir)):
    checkpoints.append([])
    for j in range(0,len(checkpoint_dir[i])): 
        checkpoint = ModelCheckpoint(filepath=checkpoint_dir[i][j], 
                            frequency = "epoch",
                            save_weights_only = False,
                             verbose= True)
        checkpoints[i].append(checkpoint)

In [5]:

for i in range(8,10):
        print(i)
        dd = df[i]
        y = dd.pop("target")
        time_id = dd.pop("time_id")
        investment_id = dd.pop("investment_id")
        data = dd.loc[: , dd.columns != 'Unnamed: 0']

        investment_ids = list(investment_id.unique())
        investment_id_size = len(investment_ids) + 1
        investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)

        with tf.device("gpu"):
            investment_id_lookup_layer.adapt(investment_id)

        model = get_model()
        model.fit([investment_id, data],y, epochs=5  ,callbacks= [checkpoints[i][0]])

8


2022-04-18 14:09:54.209279: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 14:09:54.275035: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 14:09:54.275330: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 14:09:54.279029: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/5
80794/80794 [==============================] - 1487s 18ms/step - loss: 0.8851 - mse: 0.8385 - mae: 0.6268 - mape: 101764.7891 - rmse: 0.9157

Epoch 00001: saving model to ensemle_8_model_1


2022-04-18 14:37:54.413122: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ensemle_8_model_1/assets
Epoch 2/5
80794/80794 [==============================] - 1497s 19ms/step - loss: 0.8319 - mse: 0.8291 - mae: 0.6229 - mape: 130241.8516 - rmse: 0.9106

Epoch 00002: saving model to ensemle_8_model_1
INFO:tensorflow:Assets written to: ensemle_8_model_1/assets
Epoch 3/5
80794/80794 [==============================] - 1485s 18ms/step - loss: 0.8281 - mse: 0.8268 - mae: 0.6220 - mape: 131258.9375 - rmse: 0.9093

Epoch 00003: saving model to ensemle_8_model_1
INFO:tensorflow:Assets written to: ensemle_8_model_1/assets
Epoch 4/5
80794/80794 [==============================] - 1484s 18ms/step - loss: 0.8256 - mse: 0.8248 - mae: 0.6213 - mape: 127783.7500 - rmse: 0.9082

Epoch 00004: saving model to ensemle_8_model_1
INFO:tensorflow:Assets written to: ensemle_8_model_1/assets
Epoch 5/5
80794/80794 [==============================] - 1483s 18ms/step - loss: 0.8236 - mse: 0.8232 - mae: 0.6208 - mape: 125591.7891 - rmse: 0.9073

Epoch 00005